# PyTorch Assignment: Natural Language Processing (NLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Manuel Jesús Corbacho Sánchez

### Text Classification

In Notebook 4A, we built a sentiment analysis model for movie reviews.
That particular sentiment analysis was a two-class classification problem, where the two classes were whether the review was positive or negative.
Of course, natural language comes in all sorts of different forms; sometimes we want to perform other types of classification.

For example, the AG News dataset contains text from 127600 online news articles, from 4 different categories: World, Sports, Business, and Science/Technology.
AG News is typically used for topic classification: given an unseen news article, we're interested in predicting the topic.
For this assignment, you'll be training several models on the AG News dataset.
Unlike the quick example we trained in Notebook 4A, however, we're going to *learn* the word embeddings.
Since you may be unfamiliar with AG News, we're going to walk through how to load the data, to get you started.


### Loading AG News with Torchtext

The AG News dataset is one of many included Torchtext.
It can be found grouped together with many of the other text classification datasets.
While we can download the source text online, Torchtext makes it retrievable with a quick API call&ast;. If you are running this notebook on your machine,  you can uncomment and run this block:

In [1]:
import torch
import torchtext
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
agnews_train, agnews_test = torchtext.datasets.text_classification.DATASETS["AG_NEWS"](root="./datasets")

120000lines [00:04, 26245.48lines/s]
120000lines [00:08, 14909.03lines/s]
7600lines [00:00, 14933.29lines/s]


In [2]:
# import torch
# print(torch.cuda.is_available() )


<font size="1">&ast;At the time this notebook was created, Torchtext contains a small bug in its csv reader.
You may need to change one line in the source code, as suggested [here](https://discuss.pytorch.org/t/one-error-about-the-utils-pys-code/53885) to successfully load the AG News dataset.
</font>

Unfortunately, Torchtext assumes we have network connectivity. If we don't have network access, such as notebooks running in Coursera Labs, we need to reimplement some Torchtext functionality. Skip this next block if you were able to successfully run the previous code:

In [3]:
# import torchtext
# ngrams = 1
# train_csv_path = './datasets/ag_news_csv/train.csv'
# test_csv_path = './datasets/ag_news_csv/test.csv'
# vocab = torchtext.vocab.build_vocab_from_iterator(
#     torchtext.datasets.text_classification._csv_iterator(train_csv_path, ngrams))
# train_data, train_labels = torchtext.datasets.text_classification._create_data_from_iterator(
#         vocab, torchtext.datasets.text_classification._csv_iterator(train_csv_path, ngrams, yield_cls=True), False)
# test_data, test_labels = torchtext.datasets.text_classification._create_data_from_iterator(
#         vocab, torchtext.datasets.text_classification._csv_iterator(test_csv_path, ngrams, yield_cls=True), False)
# if len(train_labels ^ test_labels) > 0:
#     raise ValueError("Training and test labels don't match")
# agnews_train = torchtext.datasets.TextClassificationDataset(vocab, train_data, train_labels)
# agnews_test = torchtext.datasets.TextClassificationDataset(vocab, test_data, test_labels)

Let's inspect the first data example to see how the data is formatted:

In [4]:
print(agnews_train[0])

(2, tensor([  432,   426,     2,  1606, 14839,   114,    67,     3,   849,    14,
           28,    15,    28,    16, 50726,     4,   432,   375,    17,    10,
        67508,     7, 52259,     4,    43,  4010,   784,   326,     2]))


We can see that Torchtext has each example as a tuple, with the first element being the label (0, 1, 2, or 3), and the second element the text data.
Notice that the text is already "tokenized": the words of the news article have been represented as word IDs, with each number corresponding to a unique word.

In previous notebooks, we've used `DataLoader`s to handle shuffling and batching.
However, if we directly try to feed these dataset objects into a `DataLoader`, we will face an error when we try to draw our first batch.
Can you figure out why?
Here's a hint:

In [5]:
print("Length of the first text example: {}".format(len(agnews_train[0][1])))
print("Length of the second text example: {}".format(len(agnews_train[1][1])))

Length of the first text example: 29
Length of the second text example: 42


Because each example is a news snippet, they can vary in length.
This is natural, as humans don't stick to consistent sentence length while writing.
This creates a bit of a problem while batching, as default tensors expect the size of each dimension to be consistent.

How do we fix this?
The common solution is to perform padding and/or truncation, picking a maximum sequence length $L$.
Inputs longer than the maximum length are truncated (i.e. $x_{t>L}$ are discarded), and shorter sequences have zeros padded to the end until they are all of length of $L$.
We'll focus on padding here, for simplicity.

We can perform this padding manually, but Pytorch has this functionality implemented.
As an example, let's pad the first two sequences to the same length:

In [6]:
from torch.nn.utils.rnn import pad_sequence

padded_exs = pad_sequence([agnews_train[0][1], agnews_train[1][1]])
print("First sequence padded: {}".format(padded_exs[:,0]))
print("First sequence length: {}".format(len(padded_exs[:,0])))
print("Second sequence padded: {}".format(padded_exs[:,1]))
print("Second sequence length: {}".format(len(padded_exs[:,1])))

First sequence padded: tensor([  432,   426,     2,  1606, 14839,   114,    67,     3,   849,    14,
           28,    15,    28,    16, 50726,     4,   432,   375,    17,    10,
        67508,     7, 52259,     4,    43,  4010,   784,   326,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])
First sequence length: 42
Second sequence padded: tensor([15875,  1073,   855,  1311,  4251,    14,    28,    15,    28,    16,
          930,   798,   321, 15875,    99,     4, 27658,    29,     6,  4460,
           12,   565, 52791,     9, 80618,  2126,     8,     3,   526,   242,
            4,    29,  3891, 82815,  6575,    11,   207,   360,     7,     3,
          127,     2])
Second sequence length: 42


Although originally of unequal lengths, both sequences are now the same length, with the shorter one padded with zeros.

We'd like the `DataLoader` to perform this padding operation as part of its batching process, as this will allow us to effectively combine varying-length sequences in the same input tensor.
Fortunately, `Dataloader`s let us override the default batching behavior with the `collate_fn` argument.

In [7]:
import numpy as np
import torch

def collator(batch):
    labels = torch.tensor([example[0] for example in batch])
    sentences = [example[1] for example in batch]
    data = pad_sequence(sentences)
    real_length = [len(example[1]) for example in batch]
    return [data, labels], real_length

Now that we have our collator padding our sequences, we can create our `DataLoader`s.
One last thing we need to do is choose a batch size for our `DataLoader`.
This may be something you have to play around with.
Too big and you may exceed your system's memory; too small and training may take longer (especially on CPU).
Batch size also tends to influence training dynamics and model generalization.
Fiddle around and see what works best.

In [8]:
BATCH_SIZE = 256

train_loader = torch.utils.data.DataLoader(agnews_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator)
test_loader = torch.utils.data.DataLoader(agnews_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)

### Simple Word Embedding Model

First, let's try out the Simple Word Embedding Model (SWEM) that we built in Notebook 4A on the AG News dataset.
Unlike before though, instead of loading pre-trained embeddings, let's learn the embeddings from scratch.
Before we begin, it will be helpful to define a few more hyperparameters.

In [9]:
VOCAB_SIZE = len(agnews_train.get_vocab())
EMBED_DIM = 100
HIDDEN_DIM = 64
NUM_OUTPUTS = len(agnews_train.get_labels())
NUM_EPOCHS = 3

Once again, we're going to organize our model as a `nn.Module`.
Instead of assuming the input is already an embedding, we're going to make learning the embedding as part of our model.
We do this by using `nn.Embedding` to perform an embedding look-up at the beginning of our forward pass.
Once we've done the look up, we'll have a minibatch of embedded sequences of dimension $L \times$ `BATCH_SIZE` $\times$ `EMBED_DIM`.
For SWEM, remember, we take the mean&ast; across the length dimension to get an average embedding for the sequence.

<font size="1"> 
&ast;Note: Technically we should only take the mean across the embeddings at the positions corresponding to "real" words in our input, and not for the zero paddings we artificially added.
This can be done by generating a binary mask while doing the padding to track the "real" words in the input.
Ultimately though, this refinement doesn't have much impact on the results for this particular task, so we omit it for simplicity.
</font>

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch

class SWEM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, num_outputs):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)        
        self.fc1 = nn.Linear(embedding_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)

    def forward(self, x):
        embed = self.embedding(x)
        embed_mean = torch.mean(embed, dim=0)
        
        h = self.fc1(embed_mean)
        h = F.relu(h)
        h = self.fc2(h)
        return h

With the model defined, we can instantiate, train, and evaluate.
Try doing so below!
Because of the way we organized our model as an `nn.Module` and our data pipeline with a `DataLoader`, you should be able to use much of the same code as we have in other examples.

<font size="1"> 
Note: Depending on you system, training may take up to a few hours, depending on how many training epochs you set.
To see results sooner, you can train for less iterations, but perhaps at the cost of final accuracy.
On the other hand, using a GPU (and GPU-enabled PyTorch) should enable full training in a couple minutes.
</font>

In [11]:
#instantiate the model in its own cell to avoid to train in several rounds
model = SWEM(vocab_size=VOCAB_SIZE, embedding_size=EMBED_DIM, hidden_dim=HIDDEN_DIM, num_outputs=NUM_OUTPUTS).to(device)
pytorch_total_params_SWEM = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params_SWEM)

9587924


In [29]:
### YOUR CODE HERE ###

from tqdm.notebook import tqdm, trange

## Training
# Instantiate model

# Binary cross-entropy (BCE) Loss and Adam Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Iterate through train set minibatchs 
for epoch in trange(NUM_EPOCHS):
    correct = 0
    num_examples = 0
    for inputs, labels in (train_loader):
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        y = model(inputs.to(device))
        loss = criterion(y.to(device), labels.to(device))
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        predictions = torch.argmax(y,dim=1)
        correct += torch.sum((predictions == labels.to(device)).float())        
        num_examples += len(labels)        
    # Print training progress
    if epoch % 25 == 0:
        acc = correct/num_examples
        print("Epoch: {0} \t Train Loss: {1} \t Train Acc: {2}".format(epoch, loss, acc))

## Testing
correct = 0
num_test = 0

with torch.no_grad():
    # Iterate through test set minibatchs 
    for inputs, labels in (test_loader):
        # Forward pass
        y = model(inputs.to(device))        
        predictions =  torch.argmax(y,dim=1).to(device)
        correct += torch.sum((predictions == labels.to(device)).float())
        num_test += len(labels) 
print('Test accuracy: {}'.format(correct/num_test))


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0 	 Train Loss: 0.4363650381565094 	 Train Acc: 0.6415833234786987
Test accuracy: 0.8840789198875427


### RNNs

SWEM takes a mean over the time dimension, which means we're losing any information about the order of the data sequence.
How detrimental is this for document topic classification?
Modify the SWEM model to use an RNN instead.
Once you get an RNN working, try a GRU and LSTM as well.

In [66]:
### YOUR CODE HERE ###
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, num_outputs):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)        
        self.rnn1 = nn.RNN(embedding_size, hidden_dim, num_layers=1, batch_first=True)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)

    def forward(self, x, real_seq_length):
        embed = self.embedding(x)
        embed = pack_padded_sequence(embed, real_seq_length, enforce_sorted=False)
        out, h_T = self.rnn1(embed)      
        out = self.fc2(h_T.squeeze(0))
        return out

    
    
class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, num_outputs):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)        
        self.GRU = nn.GRU(embedding_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)

    def forward(self, x, real_seq_length):
        embed = self.embedding(x)
        embed = pack_padded_sequence(embed, real_seq_length, enforce_sorted=False)
        out, h_T = self.GRU(embed)
        out = self.fc2(h_T.squeeze(0))
        return out

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, num_outputs):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)        
        self.LSTM = nn.LSTM(embedding_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)
        

    def forward(self, x, real_seq_length):
        embed = self.embedding(x)
        embed = pack_padded_sequence(embed, real_seq_length, enforce_sorted=False)
        out, h_T = self.LSTM(embed)
        output, (hn, cn) = self.LSTM(embed)
        out = self.fc2(cn.squeeze(0))
        return out





In [73]:
# model = RNN(vocab_size=VOCAB_SIZE, embedding_size=EMBED_DIM, hidden_dim=HIDDEN_DIM, num_outputs=NUM_OUTPUTS).to(device)
model = GRU(vocab_size=VOCAB_SIZE, embedding_size=EMBED_DIM, hidden_dim=HIDDEN_DIM, num_outputs=NUM_OUTPUTS).to(device)
# model = LSTM(vocab_size=VOCAB_SIZE, embedding_size=EMBED_DIM, hidden_dim=HIDDEN_DIM, num_outputs=NUM_OUTPUTS).to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

9613332


In [70]:
from tqdm.notebook import tqdm, trange
## Training
def trainingRNN(model_, train_loader_, learning_rate=0.001, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Iterate through train set minibatchs 
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in trange(num_epochs):
        correct = 0
        num_examples = 0
        for (inputs, labels), real_sentences_lengths in (train_loader_):
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y = model_(inputs.to(device), real_sentences_lengths)
            loss = criterion(y.to(device), labels.to(device))

            # Backward pass
            loss.backward()
            optimizer.step()

            predictions = torch.argmax(y,dim=1)
            correct += torch.sum((predictions == labels.to(device)).float())        
            num_examples += len(labels)        
        # Print training progress
        if epoch % 1 == 0:
            acc = correct/num_examples
            print("Epoch: {0} \t Train Loss: {1} \t Train Acc: {2}".format(epoch, loss, acc))

            
## Testing
def testRNN(model_, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    correct = 0
    num_test = 0
    with torch.no_grad():
        # Iterate through test set minibatchs 
        for (inputs, labels), real_sentences_lengths in (test_loader):
            # Forward pass
            y = model_(inputs.to(device), real_sentences_lengths)      
            predictions =  torch.argmax(y,dim=1).to(device)
            correct += torch.sum((predictions == labels.to(device)).float())
            num_test += len(labels) 
    print('Test accuracy: {}'.format(correct/num_test))

In [71]:
trainingRNN(model, train_loader, 0.001, 5)
testRNN(model, test_loader)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 	 Train Loss: 0.16578592360019684 	 Train Acc: 0.9551500082015991
Epoch: 1 	 Train Loss: 0.050769492983818054 	 Train Acc: 0.9644416570663452
Epoch: 2 	 Train Loss: 0.11717065423727036 	 Train Acc: 0.9702916741371155
Epoch: 3 	 Train Loss: 0.09382760524749756 	 Train Acc: 0.9778583645820618
Epoch: 4 	 Train Loss: 0.05765441060066223 	 Train Acc: 0.9804583191871643
Test accuracy: 0.8980262875556946


### Short Answer:

1\. How do the RNN, GRU, and LSTM compare to SWEM for AG News topic classification?
Are you surprised?
What about classification might make SWEM so effective for topic classification?

`Their performance is similar for the same amount of training. Yes, it is a surprise because there are a lot of differences between models. Maybe SWEM work better in topic classification because SWEM ignores the order of the words, which for topic classification maybe will not have a huge impact.`

2\. How many learnable parameters do each of the models you've trained have?

|Model |Learnable Parameters  |
|:--- | ---: |
|SWEM|9587924|
|RNN| 9592084 |
|GRU| 9613332 |
|LSTM| 9623956 |

`[Your answer here]`         `